In [1]:
import re
import os
import csv
import json
import requests
from urllib import parse
from bs4 import BeautifulSoup
from locals.output_data import exportData
from locals.date_scrape import getDate, lastStamp, searchDate
from locals.automate import By, Keys, EC, Action, wait_for, genBrowser, driverWait

In [2]:
header_list = [
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0'},
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'},
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/114.0'},
    {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 OPR/100.0.0.0'},
    {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
]

In [ ]:
def get_threads(pages):
    session = requests.Session()
    thread_url_list = []

    for page in pages:
        url = f"https://www.forexfactory.com/forum/137-interactive-trading?sort=lastpost&order=desc&page={page}"
        print(f"getting thread list from page: {page}")
        
        res = session.get(url, headers=header_list[0])

        if res.status_code == 200:
            soup = BeautifulSoup(res.text, 'lxml')

            thread_tags = soup.select('table#threadslist tr.threadlist__row div.threadlist__title-wrapper')

            for thread in thread_tags:
                thread_name = thread.get_text(strip=True)
                thread_url = thread.a['href']

                thread_dict = {
                    "thread_name": thread_name,
                    "thread_url": thread_url
                }

                thread_url_list.append(thread_dict)
        else:
            print(f"Bad Response: {res.status_code}")

    return thread_url_list

In [ ]:
def main():
    pages = get_forum_pages()

    thread_url_list = get_threads(pages)

    return thread_url_list

In [ ]:
thread_list = main()
print(thread_list)

In [ ]:
sample_url = "https://www.forexfactory.com/forum/137-interactive-trading"

In [ ]:
res = requests.get(sample_url, headers=header_list[0])
print(res.status_code)

In [ ]:
soup = BeautifulSoup(res.text, 'lxml')
page_num_tag = soup.select_one('div.forumdisplay__footer li.visible-mv a.last')
page_num = page_num_tag.string
url_path = page_num_tag['href']
actual_url = parse.urljoin(base_url, url_path)
print(page_num)
print(actual_url)

In [ ]:
# https://www.forexfactory.com/forum/137-interactive-trading

In [3]:
def get_forum_pages():
    url = input("Enter Category Url: ")
    base_url = "https://www.forexfactory.com"

    session = requests.Session()

    res = session.get(url, headers=header_list[0])

    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'lxml')
        page_num_tag = soup.select_one('div.forumdisplay__footer li.visible-mv a.last')
        page_num = page_num_tag.string
        url_path = page_num_tag['href']
        actual_url = parse.urljoin(base_url, url_path)

        return (page_num, actual_url)
        
    else:
        print(f"Bad Response of Forum Pages: {res.status_code}")
        return None

In [9]:
page_num, thread_page_url = get_forum_pages()

In [10]:
print(page_num, thread_page_url)

15 https://www.forexfactory.com/forum/137-interactive-trading?sort=lastpost&order=desc&page=15


In [14]:
# parse.urlparse(thread_url)._asdict()

url_obj = parse.urlparse(thread_page_url)
print(url_obj)
url_obj._asdict()

ParseResult(scheme='https', netloc='www.forexfactory.com', path='/forum/137-interactive-trading', params='', query='sort=lastpost&order=desc&page=15', fragment='')


{'scheme': 'https',
 'netloc': 'www.forexfactory.com',
 'path': '/forum/137-interactive-trading',
 'params': '',
 'query': 'sort=lastpost&order=desc&page=15',
 'fragment': ''}

In [21]:
help(parse.urlparse(thread_page_url))

Help on ParseResult in module urllib.parse object:

class ParseResult(ParseResult, _NetlocResultMixinStr)
 |  ParseResult(scheme, netloc, path, params, query, fragment)
 |  
 |  Method resolution order:
 |      ParseResult
 |      ParseResult
 |      builtins.tuple
 |      _NetlocResultMixinStr
 |      _NetlocResultMixinBase
 |      _ResultMixinStr
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  geturl(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  _encoded_counterpart = <class 'urllib.parse.ParseResultBytes'>
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from ParseResult:
 |  
 |  __getnewargs__(self)
 |      Return self as a plain tuple.  Used by copy and pickle.
 |  
 |  __repr__(self)
 |      Return a nicely formatted representation string
 |  
 |  _asdict(self)
 |      Return a new dict which maps field names to their v

In [7]:
forum_page_num = int(page_num) + 1
forum_pages = [page for page in range(1, forum_page_num)]
print(forum_pages)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [25]:
for page in forum_pages:
    url_obj = parse.urlparse(thread_page_url)
    url_dict = url_obj._asdict()
    bas_url = base_url = "https://www.forexfactory.com"
    path = url_dict['path']

    new_url = base_url + path + f"?sort=lastpost&order=desc&page={page}"
    print(new_url)

https://www.forexfactory.com/forum/137-interactive-trading?sort=lastpost&order=desc&page=1
https://www.forexfactory.com/forum/137-interactive-trading?sort=lastpost&order=desc&page=2
https://www.forexfactory.com/forum/137-interactive-trading?sort=lastpost&order=desc&page=3
https://www.forexfactory.com/forum/137-interactive-trading?sort=lastpost&order=desc&page=4
https://www.forexfactory.com/forum/137-interactive-trading?sort=lastpost&order=desc&page=5
https://www.forexfactory.com/forum/137-interactive-trading?sort=lastpost&order=desc&page=6
https://www.forexfactory.com/forum/137-interactive-trading?sort=lastpost&order=desc&page=7
https://www.forexfactory.com/forum/137-interactive-trading?sort=lastpost&order=desc&page=8
https://www.forexfactory.com/forum/137-interactive-trading?sort=lastpost&order=desc&page=9
https://www.forexfactory.com/forum/137-interactive-trading?sort=lastpost&order=desc&page=10
https://www.forexfactory.com/forum/137-interactive-trading?sort=lastpost&order=desc&page=